# Aiera Assistant

This notebook defines the Aiera assistant to be registered with OpenAI.

The Aiera Assistant is able to:
1. Perform event summarization
2. Perform SWOT analysis on an event
3. Extract KPIs from the event
4. Compare KPIs across historical earnings calls

## Requirements

* OpenAI API account
* Aiera API subscription (contact `sales@aiera.com`)

## 1. Ensure environment is configured
Before running the code in this notebook, ensure you've defined the `OPENAI_API_KEY` and `OPENAI_ORG_ID`.

In [ ]:
import os
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
OPENAI_ORG_ID = os.environ["OPENAI_ORG_ID"]

## 2. Define our actions
We define two json actions to collect data from Aiera's REST API. More details on how these actions are defined may be found [here](https://cookbook.openai.com/examples/assistants_api_overview_python). 
1. get_events
2. upload_event_transcripts

In [ ]:
get_events = {
  "name": "get_events",
  "description": "Retrieves a events that match the parameters provided.",
  "parameters": {
    "type": "object",
    "properties": {
      "modified_since": {
        "type": "string",
        "description": "Returns events modified since this date."
      },
      "bloomberg_ticker": {
        "type": "string",
        "description": "Filter events to 1+ bloomberg tickers (comma-separated) including the country code."
      },
      "event_type": {
        "type": "string",
        "enum": [
          "earnings",
          "earnings_release",
          "presentation",
          "investor_meeting",
          "special_situation"
        ],
        "description": "Filter by earnings, earnings_release, presentation, investor_meeting, special_situation (comma-separated)"
      },
      "start_date": {
        "type": "string",
        "description": "Fetch events on or after this date (defaults to 2 weeks ago)"
      },
      "end_date": {
        "type": "string",
        "description": "Fetch events on or before this date (defaults to 2 weeks from now)"
      }
    },
    "required": [
      "event_type",
      "start_date"
    ]
  }
}

In [ ]:
upload_event_transcripts = {
  "name": "upload_event_transcripts",
  "description": "A function that takes in a list of event_ids and uploads the event transcripts as .json files.",
  "parameters": {
    "type": "object",
    "properties": {
      "event_ids": {
        "type": "array",
        "description": "The event_ids for which to fetch transcripts",
        "items": {
          "type": "integer"
        }
      }
    },
    "required": [
      "event_ids"
    ]
  }
}

## 3. Define instructions
Instructions describe how the assistant should behave. 

In [ ]:
instructions = """You are Aiera, an expert in analyzing earnings call transcripts \
with a focus on accuracy and detail. Your role is to analyze earnings call events \
for specific companies. 

Instructions for collecting events:
1. When a user requests analysis, like 'Compare KPIs across Microsoft's 2022 \
earnings calls', use the get_events operation to retrieve relevant 'earnings' \
events. Infer the Bloomberg ticker with country code from the user's context. \
Set the start_date to November of the previous year and the end_date to today \
to ensure coverage of the fiscal period. When the user asks about the most \
recent event, always ensure you use the event occurring closest to today's \
date by calling the get_events operation.
2. Apply a stringent filter on the events fiscal_year to ensure only events \
matching the fiscal year provided by the user are included when uploading transcripts.  
3. Ask clarifying questions to determine which event to use in the case of \
ambiguity and collect new events if applicable.  
4. Upload event content using the upload_event_transcripts operation. 

Instructions for analysis:
Present your analysis, emphasizing key insights and metrics from the earnings calls. 
After each task, offer further in-depth exploration or different event analysis, \
and suggest contacting sales@aiera.com for more information.

Instructions for additional questions:
Ask clarifying questions to determine which event to use in the case of ambiguity \
and collect new events if applicable.

When suggesting analysis, propose that the user explores topics for the event, \
build a swot analysis for the company, or summarize the event.
"""

## 3. Connect to OpenAI and create assistant

In [ ]:
from openai import OpenAI
import json

# helper function for better json printing w/ openai objects
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [ ]:
client = OpenAI(
    organization = OPENAI_ORG_ID,
    api_key = OPENAI_API_KEY
)

assistant = client.beta.assistants.create(
    name="Aiera Assistant",
    instructions=instructions,
    model="gpt-4-1106-preview",
    tools = [{"type": "retrieval"}, # for file retrieval
             {"type" : "function", "function": get_events},
             {"type" : "function", "function": upload_event_transcripts}
            ]
)

show_json(assistant)

## 4. Update environment & run app
You can now exit this notebook. After, you'll need to add the assistant id to your environment:

```bash
export OPENAI_ASSISTANT_ID={your id}
```

You can find your id by running the below cell:

In [ ]:
assistant.id